In [1]:
import sqlite3
import pybedtools
import sys
import glob
import os
sys.path.append('/home/manager/Documents/ngsscriptlibrary/')
import ngsscriptlibrary as nsl
DB = 'captures.sqlite'
NGST = '/home/manager/Documents/ngstargets/'

In [2]:
conn = sqlite3.connect(DB)
c = conn.cursor()

In [3]:
def merge_bed(bedfile):
    "Merge overlaping regions in bedfile. Write bedfile with merged regions"
    bed = pybedtools.BedTool(bedfile)
    bed = bed.sort()
    bed_merged = bed.merge()
    bed_merged.saveas(bedfile)

In [4]:
for bed in glob.glob('../*/*.bed'):
    merge_bed(bed)

In [5]:
c.execute('SELECT * FROM genesis')
out = c.fetchall()

In [20]:
def parse_bed(bedfn):
    "Read BED. Return list of lists."
    bed = list()

    with open(bedfn, 'r') as f:
        lines = [line.rstrip().split() for line in f]
    lines = list(line for line in lines if line)
    lines = sorted(lines, key=lambda x: (x[0], int(x[1]), int(x[2])))

    for line in lines:
        chromosome, start, end, gene = line
        bed.append([chromosome, start, end, gene])

    return bed


def write_picard_file(fn, bed_annotated, cap):
    with open(fn, 'w') as f:
        for line in nsl.get_picard_header():
            f.write(line)
        for line in bed_annotated:
            chromosome, start, end, gene = line
            f.write('{}\t{}\t{}\t+\t{}\n'.format(chromosome, start, end, cap))
            
def write_bed_annotated(bed_annotated, fn):            
    with open(fn, 'w') as f:
        for line in bed_annotated:
            chromosome, start, end, gene = line
            f.write('{}\t{}\t{}\t{}\n'.format(chromosome, start, end, gene))

def write_varcaltargets(genes, fn, annotator):
    with open(fn, 'w') as f:
        for g in genes:
            for line in annotator.get_region(g):
                chromosome, start, end, *_ = line
                f.write('{}\t{}\t{}\t{}\n'.format(chromosome, start, end, g))
                
def write_sangertargets(genes, fn, bed_annotated):
    with open(fn, 'w') as f:
        for line in bed_annotated:
            chromosome, start, end, gene = line
            if gene in genes:
                f.write('{}\t{}\t{}\t{}\n'.format(chromosome, start, end, gene))



In [28]:
vcaps = list()
for _ in out:
    genesis, capture, pakket, panel, cnvs, cnvd, moza = _
    if panel == 'None':
        panel = None
        
    if pakket == 'PCO':
        continue
    T = nsl.TargetDatabase(DB)
    for cap in T.get_all_captures_for_genesis(genesis):
        for vcap in T.get_all_versions_for_capture(cap):
            bed = '{}/captures/{}_target.bed'.format(NGST, vcap)
            genes = T.get_genes_for_vcapture(vcap)

            TA = nsl.TargetAnnotation(bed, host='localhost', user='manager',
                                      db='annotation', genes=genes)

            bed_annotated = TA.annotate_bed_and_filter_genes()
            genes_in_bed = TA.get_genes_in_annotated_bed(bed_annotated)
            not_requested = TA.get_genes_not_requested(genesout=genes_in_bed)
            not_found = TA.get_genes_not_found(bedgenes=genes_in_bed)
            bed_annotated_fn = '{}/captures/{}_target.annotated'.format(NGST, vcap)
            picard_file_fn = '{}/captures/{}_target.interval_list'.format(NGST, vcap)

            write_bed_annotated(bed_annotated, bed_annotated_fn)
            write_picard_file(picard_file_fn, bed_annotated, vcap)
            
            if capture == pakket:
                var_target_fn = '{}/captures/{}_generegions.bed'.format(NGST, vcap)
                    write_varcaltargets(genes, var_target_fn, TA)

                print(vcap, not_found, not_requested)
            for pakket in T.get_all_pakketten_for_genesis(genesis):
                vpakketten = T.get_all_versions_for_pakket(pakket)
                for vpakket in vpakketten:
                    vpakketten_info = T.get_all_info_for_vpakket(vpakket)
                    for vpakket_info in vpakketten_info:
                        vpakket_grootte, vpakket_genen = vpakket_info
                    if capture != pakket:
                        var_target_fn = '{}/pakketten/{}_generegions.bed'.format(NGST, vpakket)
                            write_varcaltargets(vpakket_genen, var_target_fn, TA)

                        for vpakket_gen in vpakket_genen:
                            if vpakket_gen not in genes:
                                print(genesis, vcap, vpakket, vpakket_gen)

                    for vpanel in T.get_all_panels_for_genesis(genesis):
                        vpanels = T.get_all_versions_for_panel(panel)
                        for vpanel in vpanels:
                            vpanel_infos = T.get_all_info_for_vpanel(vpanel)
                            for vpanel_info in vpanel_infos:
                                vpanel_grootte, vpanel_genen = vpanel_info
                                for vpanel_gen in vpanel_genen:
                                    if vpanel_gen not in genes:
                                        print(genesis, vcap, vpakket, vpanel, vpanel_gen)
    vcap = T.get_current_capture(genesis)
    vpakket = T.get_current_pakket(genesis)
    vpanel = T.get_current_panel(genesis)

    try:
        bed_annotated = parse_bed(os.path.join(NGST, 'captures', '{}_target.annotated'.format(vcap)))
    except ValueError as e:
        print('ERROR', e)
    
    if panel == 'None':
        panel = None
    
    if panel is None and capture != pakket:
        sangers = 'pakketten/{}_target.bed'.format(vpakket)
        genes = T.get_genes_for_vpakket(vpakket)
    elif panel is None and capture == pakket:
        sangers = 'captures/{}_target.bed'.format(vpakket)
        genes = T.get_genes_for_vpakket(vpakket)
    elif panel is not None:
        print(genesis, vpanel)
        vpanel = vpanel.replace('typeA', '')
        genes = T.get_genes_for_vpanel(vpanel)
        sangers = 'panels/{}typeAv{}_target.bed'.format(vpanel.split('v')[0], vpanel.split('v')[1])

    sangers = os.path.join(NGST, sangers)
    bed_annotated_fn = '{}/captures/{}_target.annotated'.format(NGST, vcap)
    if not os.path.isfile(sangers):
        print('ERROR', sangers)
        
    write_sangertargets(genes, sangers, bed_annotated)   

/home/manager/Documents/ngstargets/captures/NEUROv4_target.annotated
ERROR need more than 3 values to unpack
ALS ALStypeAv2
/home/manager/Documents/ngstargets/captures/NEUROv4_target.annotated
ERROR need more than 3 values to unpack
LIMGIR OVRtypeAv1
/home/manager/Documents/ngstargets/captures/NEUROv4_target.annotated
ERROR need more than 3 values to unpack
NEUROP CMTtypeAv3
/home/manager/Documents/ngstargets/captures/NEUROv4_target.annotated
ERROR need more than 3 values to unpack
PCH PCHtypeAv2
